# Earth Moon Example

In this notebook, we show a simple example on how to create a P1ON animation file using python.
The animation will show the moon orbiting around the earth.

In [1]:
#Make sure you have p1on installed
#(uncomment command if needed)
#!pip install -e ..

First let's import all the necessary packages.

In [2]:
import numpy as np
import p1on

Let's start with creating two spheres: one for Earth and one for the Moon

Each of the spheres were given a name. This allows us latter to easily identify both of them in P1ON's editor.\
We also added descriptions. Descriptions are texts which will be shown in the animation if Earth or the Moon are selected. \
The other two properties, radius and color are exactly what you think they are.

In [3]:
earth = p1on.Sphere('Earth', description="That's our planet!", radius=1.0, color='blue')
moon = p1on.Sphere('Moon', description='Our oldest pal!', radius=0.27, color='grey')

Right now nothing would happen in the animation. Even worse, the moon is inside the Earth! \
To prevent this doomsday scenario, let's create a path for the Moon to follow.

Paths are simply a list of control points, assigning distinct moments in time a fixed position in space. \
A simple numpy array (or anything that can be converted into one using np.array) is sufficient. \
It only has to consist of 4 columns (i.e. shape of (N,4)). These columns are time, followed by the x, y and z coordinates.

In [4]:
#We make the orbit a bit smaller to make the animation easier to watch
r = 5.0
t = np.linspace(0, 2*np.pi)
x = r * np.cos(t)
y = r * np.sin(t)
z = np.zeros_like(t)

path = np.column_stack((t, x, y, z))
print(path.shape)

(50, 4)


We just introduced time as a variable. Since the usual regime P1ON was designed for is particle physics, \
time is usually denoted in nanoseconds. However, from the animation perspective this is purely arbitrary. \
In fact, as we later see we can set the speed of the animation ourselves.

Next we have to tell the Moon to follow its path we just created for him. \
This can be done by simply assigning the numpy array to the corresponding property:

In [5]:
moon.position = path

Internally, P1ON will create a corresponding `Path` object during serialization with a name based on the object and the property its assigned to.\
We can do that ourselves if we want to give it a more meaningful name.

In [6]:
orbit = p1on.Path(path, name='Moon Orbit')
moon.position = orbit

We're almost done. Now we have to bundle all things up.\
This happens in the `Project` class:

In [7]:
project = p1on.Project('Earth Moon',
    author='p1on',
    description='The Moon orbiting around the Earth.',
    speedRatio=2*np.pi / 5.0,
    animatables=[earth, moon])

The `Project` class is pretty straight forward: We can give it a name and optionally name the author and add a description. \
Here you can also specify the animation speed in time units (usually nanoseconds) per second. Here the Moon will complete one revolution in 5 seconds. \
The most important thing here is to list all objects (called `animatables`) we want to include in the animation.

If you look in the API, you'll know that `Project` allows to list paths to include. Any path that is referenced by any of the animatibles, will be included automatically. \
This is indented to allow to add additional information or data to manipulate inside P1ON. The same applies to the other types of data.

There are also some more advanced properties available, we won't look into here. For now let's just change the clear color, i.e. the background color of the animation scene
to a nice yellow for the sun.

In [8]:
project.clearColor = 'light yellow'

The only thing left to do, is to save the animation to disk:

In [9]:
p1on.saveProject(project, '../examples/earth_moon.p1on')